# Workflow
### Data preprocess
* file directory에서 데이터 불러오기
* 데이터 형식에 맞춰 DataFrame으로 만들기 (dictionary 형태 이용)
* 필요하지 않은 column 제거 및 결측치 처리
* graph구조로 만들기위해 node,edge가 될 attribute들 정하기
* long tailed 제거 (optional)
* dgl library를 이용해 grpah structure로 변환
* split train/test/validation 
* preprocessed data 저장

### Model
* TODO

### Metrics
* TODO

### Evaluation
* Metrics랑 같이넣을지 고민

### Test
* 결과값 및 분석 그래프 시각화

In [1]:
# 데이터 전처리

import os
import re
import argparse
import pickle
import pandas as pd
import numpy as np
import scipy.sparse as ssp
# import dgl
import torch
# import torchtext
from builder import PandasGraphBuilder
from data_utils import *
from copy import deepcopy

def remove_infrequent_users(data, min_counts=10):
    '''Remove infrequent users'''
    df = deepcopy(data)
    counts = df['UserId'].value_counts()
    df = df[df['UserId'].isin(counts[counts >= min_counts].index)]
    
    return df    

def remove_infrequent_items(data, min_counts=10):
    '''Remove infrequent items'''
    df = deepcopy(data)
    counts = df['ProductId'].value_counts()
    df = df[df['ProductId'].isin(counts[counts >= min_counts].index)]
    
    return df

# if __name__ == '__main__':
#     parser = argparse.ArgumentParser()
#     parser.add_argument('directory', type=str)
#     parser.add_argument('output_path', type=str)
#     args = parser.parse_args()
#     directory = args.directory
#     output_path = args.output_path

#     # Read amazon beauty dataset
#     #amazon = pd.read_csv('./amazon_beauty/ratings_Beauty.csv')
#     amazon = pd.read_csv(os.path.join(directory, 'ratings_Beauty.csv'))

#     # Separate user, item columns
#     # user_col = amazon.drop(['ProductId','Rating','Timestamp'], axis=1).astype('category')
#     # item_col = amazon.drop(['UserId','Rating','Timestamp'], axis=1).astype('category')
    

#     # Filter out user,item with fewer than 10 interactions
#     filtered_data = remove_infrequent_users(amazon, 10)
#     filtered_data = remove_infrequent_items(filtered_data, 10)
    
#     # Get distinct users and items
#     distinct_users = filtered_data['UserId'].unique()
#     distinct_items = filtered_data['ProductId'].unique()
#     users = pd.DataFrame(distinct_users, columns=['UserId'])
#     items = pd.DataFrame(distinct_items, columns=['ProductId'])

#     # Build graph
#     graph_builder = PandasGraphBuilder()
#     graph_builder.add_entities(users, 'UserId', 'user')
#     graph_builder.add_entities(items, 'ProductId', 'item')
#     graph_builder.add_binary_relations(filtered_data, 'UserId','ProductId','purchased')
#     graph_builder.add_binary_relations(filtered_data, 'ProductId', 'UserId', 'purchased-by')

#     g = graph_builder.build()

#     # Add user-item edge features
#     g.edges['purchased'].data['rating'] = torch.LongTensor(filtered_data['Rating'].values)
#     g.edges['purchased'].data['timestamp'] = torch.LongTensor(filtered_data['Timestamp'].values)
#     g.edges['purchased-by'].data['rating'] = torch.LongTensor(filtered_data['Rating'].values)
#     g.edges['purchased-by'].data['timestamp'] = torch.LongTensor(filtered_data['Timestamp'].values)

#     # Train, Test, Validation split by time
#     train_indices, val_indices, test_indices = train_test_split_by_time(filtered_data, 'Timestamp', 'UserId')

#     # Build Train Graph
#     train_g = build_train_graph(g, train_indices, 'user','item','purchased','purchased-by')
#     assert train_g.out_degrees(etype='purchased').min() > 0

#     val_matrix, test_matrix = build_val_test_matrix(g, val_indices, test_indices, 'user','item','purchased')

#     # Item text set
#     item_text_dataset = {'productid' : items['ProductId'].values}

#     dataset = {
#         'train-graph': train_g,
#         'val-matrix': val_matrix,
#         'test-matrix': test_matrix,
#         'item-texts': item_text_dataset,
#         'item-images': None,
#         'user-type': 'user',
#         'item-type': 'item',
#         'user-to-item-type': 'purchased',
#         'item-to-user-type': 'purchased-by',
#         'timestamp-edge-column': 'timestamp'}

#     with open(output_path, 'wb') as f:
#         pickle.dump(dataset, f)

#     with open('processed_beauty_filtered.pkl', 'rb') as fr:
#         data = pickle.load(fr)
#     print(data)

Using backend: pytorch


In [3]:
# movielens 

In [2]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 25.9 MB 10.7 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import os
import re
import numpy as np
from scipy.sparse.linalg import svds

directory = './ml-1m'

users = []
with open(os.path.join(directory, 'users.dat'), encoding='latin1') as f:
    for l in f:
        id_, gender, age, occupation, zip_ = l.strip().split('::')
        users.append({
            'user_id' : int(id_),
            'gender' : gender,
            'age' : age,
            'occupation' : occupation,
            'zip' : zip_,
        })
users = pd.DataFrame(users).astype('category')

movies = []
with open(os.path.join(directory, 'movies.dat'), encoding='latin1') as f:
    for l in f:
        id_, title, genres = l.strip().split('::')
        genres_set = set(genres.split('|'))

        # extract year
        assert re.match(r'.*\([0-9]{4}\)$', title)
        year = title[-5:-1]
        title = title[:-6].strip()

        data = {'movie_id': int(id_), 'title': title, 'year': year}
        for g in genres_set:
            data[g] = True
        movies.append(data)
movies = pd.DataFrame(movies).astype({'year': 'category'})

ratings = []
with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
    for l in f:
        user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
        ratings.append({
            'user_id': user_id,
            'movie_id': movie_id,
            'rating': rating,
            'timestamp': timestamp,
            })
ratings = pd.DataFrame(ratings)

ratings = ratings.drop(['timestamp'],axis=1)
movies = movies[['movie_id','title']]
df = pd.merge(ratings, movies, on='movie_id')
print(df)

# column은 movie로, row는 user로
df = df.pivot(values='rating', index='user_id', columns='movie_id').fillna(0)
print('pivot table로 변환')
print(df)

# pivot_table 값을 matrix로 변환
matrix = df.values

U, sigma, Vt = svds(matrix, k = 20)
print(U.shape, sigma.shape, Vt.shape)


         user_id  movie_id  rating                                 title
0              1      1193       5       One Flew Over the Cuckoo's Nest
1              2      1193       5       One Flew Over the Cuckoo's Nest
2             12      1193       4       One Flew Over the Cuckoo's Nest
3             15      1193       4       One Flew Over the Cuckoo's Nest
4             17      1193       5       One Flew Over the Cuckoo's Nest
...          ...       ...     ...                                   ...
1000204     5949      2198       5                           Modulations
1000205     5675      2703       3                        Broken Vessels
1000206     5780      2845       1                            White Boys
1000207     5851      3607       5                     One Little Indian
1000208     5938      2909       4  Five Wives, Three Secretaries and Me

[1000209 rows x 4 columns]
pivot table로 변환
movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
use

In [18]:
sigma = np.diag(sigma)
sigma.shape

(20, 20)

In [20]:
mf_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
df_preds = pd.DataFrame(mf_predicted_ratings, columns = df.columns)
df_preds

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,3.236188,0.691441,-0.004421,-0.013907,0.125572,-0.282993,-0.064685,0.167259,-0.100287,-0.048986,...,-0.013743,0.003684,0.039305,-0.006088,-0.073551,0.302123,-0.074654,-0.019404,-0.003208,0.069246
1,1.271428,0.424805,0.102387,0.057807,0.098287,0.738487,0.094775,0.045328,0.185549,1.402783,...,-0.045296,-0.002519,-0.003512,0.040962,-0.036473,0.157463,-0.176261,-0.022356,-0.001310,0.033546
2,1.279739,0.162583,0.102715,-0.069212,-0.021542,0.163654,-0.123966,0.013215,0.022383,0.551677,...,-0.028523,-0.002628,0.012135,0.004124,-0.029672,0.230783,-0.127389,-0.041387,-0.019541,-0.138831
3,0.258133,-0.112573,0.019786,0.060456,0.024741,0.295179,-0.006009,0.000252,-0.022433,0.080312,...,0.015833,-0.000350,-0.003163,-0.024258,-0.026182,-0.009095,0.046987,-0.019518,0.015120,-0.084392
4,1.073362,0.115839,-0.210225,0.142634,-0.228213,1.438930,-0.265505,-0.002904,-0.058044,0.265604,...,0.084047,-0.001990,-0.013753,-0.033778,0.011687,-0.011893,0.493642,0.022157,0.074390,0.168795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,3.959307,1.183006,-0.377753,0.450136,-0.222037,1.850436,0.253437,0.119271,-0.088443,0.653970,...,0.218243,0.002494,-0.011816,-0.046864,0.227337,-0.265386,0.839719,0.215358,0.205192,0.528081
6036,1.520510,0.148834,-0.177174,-0.134580,-0.028856,0.539473,-0.145528,-0.027442,-0.049522,-0.042445,...,-0.006238,-0.005405,-0.021702,-0.011869,0.040246,0.046442,0.026163,0.020420,-0.044727,0.133255
6037,0.584086,0.006551,-0.047754,-0.022483,-0.054682,-0.114331,0.102466,-0.006497,-0.013132,0.093405,...,-0.000771,-0.002040,-0.000031,-0.012474,-0.010803,0.007900,-0.057611,-0.013875,-0.004654,-0.062523
6038,0.979589,0.338221,0.205096,-0.020612,0.112932,-0.386045,0.421680,0.062016,-0.115824,0.345755,...,0.005408,0.020299,0.014549,-0.043421,0.077311,0.039059,-0.088293,0.013123,-0.007281,-0.158822


In [60]:
def recommendation(df_preds, user_id, movies, ratings, num_recommendations=10):
    user_idx = user_id - 1
    sorted_preds = df_preds.iloc[user_idx].sort_values(ascending=False)
    true_rating = ratings[ratings.user_id == user_idx]
    user_history = true_rating.merge(movies, on='movie_id').sort_values(['rating'], ascending=False)
    recommendations = movies[~movies['movie_id'].isin(user_history['movie_id'])]
    recommendations = recommendations.merge(pd.DataFrame(sorted_preds).reset_index(), on='movie_id')
    recommendations = recommendations.rename(columns={user_idx : 'Predictions'}).sort_values('Predictions', ascending=False)[:num_recommendations]
    
    return user_history, recommendations

In [61]:
rated, preds = recommendation(df_preds, 330, movies, ratings, 10)
print(rated)
print(preds)

     user_id  movie_id  rating              title
422      329       230       5  Dolores Claiborne
267      329      2294       5               Antz
687      329       541       5       Blade Runner
686      329      2959       5         Fight Club
30       329       588       5            Aladdin
..       ...       ...     ...                ...
261      329      3239       1   Isn't She Great?
260      329      1707       1       Home Alone 3
571      329      3715       1            Burglar
572      329      2840       1           Stigmata
265      329      3168       1         Easy Rider

[764 rows x 4 columns]
      movie_id                          title  Predictions
2775      3753                   Patriot, The     2.913342
629        858                 Godfather, The     2.693390
897       1221        Godfather: Part II, The     2.219790
672        908             North by Northwest     2.182542
2616      3555                          U-571     1.948591
2776      3755        

In [4]:
df

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
print(ratings)
# 어떤 비율로 표본 추출을 하고싶은지 df.sample에서 frac을 0~1 사이로 설정
q = ratings.sample(frac=1).reset_index(drop=True)
q
sparse_matrix = q.groupby('movie_id').apply(lambda x : pd.Series(x['rating'].values, index = x['user_id'])).unstack()
sparse_matrix


         user_id  movie_id  rating
0              1      1193       5
1              1       661       3
2              1       914       3
3              1      3408       4
4              1      2355       5
...          ...       ...     ...
1000204     6040      1091       1
1000205     6040      1094       5
1000206     6040       562       5
1000207     6040      1096       4
1000208     6040      1097       4

[1000209 rows x 3 columns]


user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,5.0,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def make_binary(x):
    if x>=3.5:
        x = 1
    else:
        x = 0
    return x

In [8]:
ratings

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [9]:
x = ratings.groupby('user_id').apply(lambda x : pd.Series(x['rating'].values, index = x['movie_id'])).unstack().fillna(0).to_numpy()
x.shape

(6040, 3706)

In [10]:
q['relevance'] = q['rating'].apply(make_binary)
print(q)
a = q.groupby('movie_id').apply(lambda x : pd.Series(x['relevance'].values, index=x['user_id'])).unstack().fillna(0).to_numpy().T
a

         user_id  movie_id  rating  relevance
0            671      1252       5          1
1           4459      1732       5          1
2            990        89       3          0
3           1117      1641       5          1
4           1627      1644       1          0
...          ...       ...     ...        ...
1000204     3471      1801       3          0
1000205     1335      2150       2          0
1000206     2480      2683       3          0
1000207     4375      2012       4          1
1000208     4033      1967       4          1

[1000209 rows x 4 columns]


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
idx, jdx = sparse_matrix.fillna(0).to_numpy().nonzero()
indice = list(zip(idx, jdx))
np.random.shuffle(indice)
val_set = []
set_idx = []
val_sparse_matrix = sparse_matrix
train_sparse_matrix = sparse_matrix

for i,j in indice[:ratings.shape[0] // 10]:
    val_set.append((i, j, sparse_matrix.iloc[i, j]))
    set_idx.append((i,j))
    val_sparse_matrix.iloc[i,j] = sparse_matrix.iloc[i,j]
    train_sparse_matrix.iloc[i,j] = 0
len(val_set)

100020

In [27]:
l = []
sparse_matrix = sparse_matrix.fillna(0).to_numpy()
# sparse_without_train_matrix = sparse_without_train_matrix.to_numpy()
for i,j in set_idx:
    print(i,j)
    l.append(sparse_matrix[i,j])
print(l)

3132 3897
1550 2490
1120 2905
849 1141
992 1627
2664 4714
1164 2224
1294 2675
2775 5667
581 4778
873 30
287 2495
2650 5119
935 5183
3441 980
527 3939
1288 2083
2891 4490
370 3023
1759 2543
3555 3149
356 3195
1767 1510
1463 1697
2563 1670
2096 5584
240 3442
575 548
2944 751
561 4200
1108 2826
1142 4833
1459 102
328 4351
838 3291
2201 4066
2815 3031
619 4226
1049 2898
892 5069
2803 5377
2794 5174
3590 4100
1923 1679
1278 5895
1821 4615
106 258
253 3911
3314 3086
1581 3728
2775 3846
2575 1973
2656 2765
1439 4139
2490 1732
651 2933
1485 4226
2542 5716
1775 1779
2277 5248
513 5615
3559 2444
2197 2859
839 1512
2932 2912
2708 5923
2739 1997
1778 3028
1091 3300
3189 3194
1107 2147
3005 3337
723 3996
2384 580
956 5025
1968 3843
1292 4052
3157 4280
2055 2367
2683 3588
1848 3920
2819 3694
2838 962
1964 4276
1052 795
2781 1694
1029 4519
2958 947
3024 3031
466 5658
2099 3202
117 3128
346 5247
1307 4957
3235 2480
2944 4646
1780 5758
1091 4725
1096 3716
1184 1648
1832 400
1328 2700
2005 3614
2838 255

1260 5082
2210 5025
2633 4063
1877 5916
3277 284
136 2379
2351 3975
3058 3962
992 4237
1114 2888
576 2731
865 4956
2 4728
2734 2865
3437 4456
233 1980
1485 5452
3248 3651
1482 2076
3196 1598
3197 2810
1823 3824
1202 1977
2403 273
3192 1293
2280 4084
2221 3818
2785 2164
1528 3310
1384 133
2347 4138
1314 165
844 2382
2823 1940
2959 4117
3548 481
590 4317
202 1094
309 254
2131 2265
3219 3841
678 2099
2210 4311
2292 1554
3412 2091
1371 1927
1502 3074
2656 3609
2384 4741
2882 2062
1785 1099
1516 1665
1516 3044
945 263
891 4036
18 1853
1303 1494
1007 5963
1215 2640
478 1272
2140 4435
2819 4559
1264 1796
1406 4199
702 1108
3315 1919
1239 4343
3036 3517
1613 2138
1981 2846
3020 3774
2515 5569
1817 4008
1140 2010
3701 1711
917 1606
2863 589
2511 2004
1201 5977
1986 1206
1108 2392
1201 5579
1943 2105
1249 5449
598 4561
159 1446
3317 4941
3471 5360
1455 4747
1476 551
2490 122
1537 4102
3155 2817
2332 4400
466 3719
1770 4665
2036 2750
1164 2665
2664 4321
865 1146
844 2664
3218 2287
2773 4888
1273 

454 1560
2359 15
1785 3657
2686 3417
1108 32
3520 3840
881 1882
3180 2776
2057 5633
2415 5366
1250 5318
1848 479
3405 305
1260 2245
1712 734
1224 5766
1904 5113
1732 2167
1951 5628
3228 2378
1886 4578
3099 835
573 3590
2819 4053
2668 3139
1543 4256
3126 5688
735 677
2668 1629
2238 4865
3443 3475
972 3735
786 3419
2778 4999
1435 735
1821 4337
3246 3691
2775 1004
1146 4950
370 156
1165 5876
574 5733
3317 1836
3650 1686
579 3987
1566 3556
583 3928
1897 5794
1821 748
1732 2055
3294 3848
2240 1164
1148 2095
149 1949
2592 3057
2374 3284
3550 4133
3680 116
1106 1911
1084 4982
802 3409
1883 1925
3392 5070
3398 4487
585 4487
2847 2591
2516 997
2867 1834
1773 6027
174 1732
2885 71
3487 1610
284 5462
253 2231
220 2945
3243 1341
805 5626
2312 466
2508 5305
1762 4254
1197 2303
2541 3095
76 3726
2285 5045
1383 2179
1738 2011
1732 4735
1104 5692
2078 4655
167 4185
2213 5102
2519 1358
1167 1266
420 645
593 4831
1907 1586
108 1447
1371 5606
2322 1316
2213 5349
1817 5715
1398 2009
2650 2877
2013 1337
30

1545 3449
863 1158
964 5108
443 3194
3491 5593
2497 1172
2892 4484
910 2409
3601 586
2930 3278
842 5995
1130 3184
1630 3715
2505 2785
2466 2278
2981 5469
2701 3194
742 4790
593 4297
3622 3284
1206 4052
1633 2180
2480 1643
176 5651
3275 3840
1820 3612
2012 954
3377 5579
3130 5405
523 5648
2748 934
1229 116
1786 3886
166 5248
908 5925
6 1500
106 1751
945 5110
1878 1857
2185 3483
2069 1919
3623 3597
2368 1488
1727 554
2524 2966
1054 2732
228 2484
1934 5884
579 5358
2198 3425
1428 1623
1124 3129
2354 5721
3470 2011
2124 5879
2056 1469
891 4884
178 1241
498 4655
3512 299
2508 4278
2781 3824
1110 4509
2105 5715
1 4019
2708 3223
1788 2373
2519 3068
1785 2726
2856 1500
87 3716
1178 1222
959 1523
1869 5604
1962 2670
3243 1592
572 5755
1634 3179
527 4918
1940 3200
1541 2130
687 1099
2462 5569
1175 4851
3378 1243
101 5346
1528 4353
2400 5698
3462 5622
1131 4031
737 745
692 5949
992 600
106 3291
2556 4898
1928 172
3279 4578
1120 2062
357 5542
2708 5403
745 402
3701 2587
3049 1638
2704 5349
533 320

KeyboardInterrupt: 

In [30]:
sparse_matrix[0,0]

5.0

In [93]:
val_sparse_matrix = sparse_matrix
val_sparse_matrix = val_sparse_matrix.replace([1,2,3,4,5], 0).fillna(0)
val_sparse_matrix.iloc[0,0] = sparse_matrix.iloc[0,0]
val_sparse_matrix = val_sparse_matrix.unstack().apply(make_binary).unstack()
val_sparse_matrix = val_sparse_matrix.to_numpy()
val_sparse_matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [107]:
val_del = np.delete(val_sparse_matrix, outlier_idx, axis=0)
val_del

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [106]:
relevance_num = a.sum(axis=1)
outlier_idx = np.where(relevance_num==0)
outlier_idx

(array([3597, 4485]),)

In [39]:
user_idx = np.repeat(np.arange(3),10)
#user_idx = user_idx[:,np.newaxis]
recommendations = np.array([[0,50,23,5,43,75,98,23,56,10],[0,50,23,5,43,75,98,23,56,10],[0,50,23,5,43,75,98,23,56,10]])
item_idx = recommendations.flatten()
#item_idx[:,np.newaxis]
print(user_idx.shape, item_idx.shape, a.shape)
b = a[user_idx, item_idx].reshape((3, 10))
b.sum(axis=1) / [2,3,2]

(30,) (30,) (6040, 3706)


array([0.5, 0. , 0. ])

In [42]:
x = np.array([1,5,15,20,25])
y = np.array([2,3,5,2,3])
x / y

array([ 0.5       ,  1.66666667,  3.        , 10.        ,  8.33333333])

In [5]:
import heapq
import numpy as np

rec = []
b = np.array([[2.5,0,1,4.2,4.1,5], [3.5,4.1,2,5,4.9,2.6]])
rec_idx = list(map(list(b[0]).index, heapq.nlargest(3,b[0])))
print(rec_idx)
rec.append(rec_idx)
np.array(rec).flatten()


[5, 3, 4]


array([5, 3, 4])

In [16]:
# x = ratings.groupby('user_id').apply(lambda x : pd.Series(x['rating'].values, index = x['movie_id'])).unstack().fillna(0).to_numpy()
np.set_printoptions(precision=6, suppress=True)
pd.options.display.float_format = '{:.5f}'.format
pd.reset_option('display.float_format')
history = []
#for u in range(1,6041):
    

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [38]:
f = open('movielens.txt', 'w')
df = ratings.drop('rating', axis=1)
data = {}
for i in range(len(df.index)):
    f.write(str(df.iloc[i]['user_id']))
    f.write(' ')
    f.write(str(df.iloc[i]['movie_id']))
    f.write('\n')
f.close()

In [7]:
df = ratings.drop('rating', axis=1)
df.head()

,user_id,movie_id
0,1,1193
1,1,661
2,1,914
3,1,3408
4,1,2355


In [169]:
df1 = deepcopy(df)
counts = df1['user_id'].value_counts()
df1 = df1[df1['user_id'].isin(counts[counts >= 10].index)]
df1

,user_id,movie_id
0,1,1193
1,1,661
2,1,914
3,1,3408
4,1,2355
...,...,...
1000204,6040,1091
1000205,6040,1094
1000206,6040,562
1000207,6040,1096


In [167]:
df2 = deepcopy(df1)
counts = df2['movie_id'].value_counts()
df2 = df2[df2['movie_id'].isin(counts[counts >=10].index)]

In [253]:
from sklearn.model_selection import train_test_split

#df = ratings.drop('rating', axis=1)

movie_train, movie_test = train_test_split(df2, test_size=0.2, shuffle=True, random_state=2022)
# movie_train, movie_val = train_test_split(movie_train, test_size=0.1, shuffle=True, random_state=2022)

print(len(movie_train.index))
print(len(movie_test.index))
#print(len(movie_val.index))

798831
199708


In [81]:
len(df)

1000209

In [254]:
movie_train = movie_train.sort_values('user_id',ascending=True)
movie_test = movie_test.sort_values('user_id', ascending=True)

In [255]:
movie_train = movie_train.reset_index(drop=True)
movie_test = movie_test.reset_index(drop=True)

In [326]:
movie_test

,user_id,movie_id
0,1,661
1,1,1721
2,1,1287
3,1,1022
4,1,1246
...,...,...
199703,6040,1254
199704,6040,908
199705,6040,150
199706,6040,1213


In [336]:
len(movie_test[movie_test['user_id']==2])

37

In [342]:
num = 1
f = open('real_test.txt','w')
user_idx = []
for i in range(len(movie_test)+1):
    if movie_test.iloc[i+1]['user_id'] == num and num<6040:
        user_idx.append(movie_test.iloc[i]['movie_id'])
    elif movie_test.iloc[i+1]['user_id'] != num  and num<6040:
        user_idx.append(movie_test.iloc[i]['movie_id'])
        user_idx.insert(0, num-1)
        for j in user_idx[:-1]:
            f.write(str(j))
            f.write(' ')
        f.write(str(user_idx[-1]))
        f.write('\n')
        num+=1
        user_idx = []
    if num==6040:
        break

user_idx = []
for i in range(len(movie_test[movie_test['user_id']==6040])):
    user_idx.append(movie_test[movie_test['user_id']==6040].iloc[i]['movie_id'])
user_idx.insert(0,num-1)
for k in user_idx[:-1]:
    f.write(str(k))
    f.write(' ')
f.write(str(user_idx[-1]))
f.close()
##############################################

num = 1
f = open('real_train.txt','w')
user_idx = []
for i in range(len(movie_train)+1):
    if movie_train.iloc[i+1]['user_id'] == num and num<6040:
        user_idx.append(movie_train.iloc[i]['movie_id'])
    elif movie_train.iloc[i+1]['user_id'] != num  and num<6040:
        user_idx.append(movie_train.iloc[i]['movie_id'])
        user_idx.insert(0, num-1)
        for j in user_idx[:-1]:
            f.write(str(j))
            f.write(' ')
        f.write(str(user_idx[-1]))
        f.write('\n')
        num+=1
        user_idx = []
    if num==6040:
        break

user_idx = []
for i in range(len(movie_train[movie_train['user_id']==6040])):
    user_idx.append(movie_train[movie_train['user_id']==6040].iloc[i]['movie_id'])
user_idx.insert(0,num-1)
for k in user_idx[:-1]:
    f.write(str(k))
    f.write(' ')
f.write(str(user_idx[-1]))
f.close()
            
    

In [289]:
movie_test[movie_test['user_id']==4]

,user_id,movie_id
58,4,3418
59,4,3468
60,4,2028
61,4,1097


In [279]:
movie_test.iloc[58]['user_id']

4

In [139]:
f = open('ttest.txt','r')
for l in f.readlines():
    if len(l) > 0:
        l = l.strip('\n')
        try:
            items = [int(i) for i in l.split(' ')[1:]]
        except Exception:
            continue
        print(items)
#         n_items = max(0, max(items))
#         print(n_items)

[1270, 1287, 2918, 661, 2321, 1022, 1721, 608, 1961, 48, 1246]
[2194, 318, 3468, 2312, 3893, 163, 3678, 1207, 2126, 21, 3030, 593, 3654, 1610, 3107, 3035, 165, 3105, 3108, 3068, 459, 480, 647, 1188, 1244, 1552, 1597, 1357, 442, 434, 380, 1917, 1253, 2396, 235]
[3619, 1266, 1196, 2871, 1049, 1378, 3552, 1198, 2081]
[3418, 2028, 1097]
[1175, 2858, 1191, 3624, 1643, 2355, 1653, 1554, 860, 714, 1759, 2725, 2359, 34, 2770, 1500, 1966, 1649, 896, 1243, 1715, 1466, 3514, 3113, 1392, 1794, 24, 1580, 1747, 1909, 202, 2916, 1704, 2734, 2384, 1921, 2890, 2070]
[2396, 3753, 266, 1007, 595, 838, 3408, 1380, 1043, 597, 1959, 3624, 569, 3565, 377, 3536, 1028]
[110, 2028, 861, 1580, 3793, 1573, 3418]
[1488, 24, 105, 480, 1840, 2427, 3260, 3250, 3246, 150, 2028, 296, 3418, 2324, 1701, 1589, 2006, 733, 1265, 2916, 253, 16, 3147]
[1310, 2324, 608, 3298, 1060, 597, 1721, 745, 3006, 3623, 527, 1358, 2959, 994, 2571, 1343, 3826, 529, 1221, 1, 1500]
[1270, 1513, 2657, 743, 2109, 1210, 1197, 3812, 3608, 2082,

In [42]:
f = open('train.txt', 'w')
for i in range(len(movie_train.index)):
    f.write(str(movie_train.iloc[i]['user_id']))
    f.write(' ')
    f.write(str(movie_train.iloc[i]['movie_id']))
    f.write('\n')
f.close()

f = open('test.txt', 'w')
for i in range(len(movie_test.index)):
    f.write(str(movie_test.iloc[i]['user_id']))
    f.write(' ')
    f.write(str(movie_test.iloc[i]['movie_id']))
    f.write('\n')
f.close()

f = open('valid.txt', 'w')
for i in range(len(movie_val.index)):
    f.write(str(movie_val.iloc[i]['user_id']))
    f.write(' ')
    f.write(str(movie_val.iloc[i]['movie_id']))
    f.write('\n')
f.close()

In [9]:
for i in range(5):
    print(df.iloc[i]['user_id'],' ',df.iloc[i]['movie_id'])

1   1193
1   661
1   914
1   3408
1   2355


In [49]:
f = open('movielens/valid.txt', 'r')
cf = np.loadtxt(f, dtype=np.int32)
n_items = max(cf[:,1])
n_items


3952

In [32]:
d = {}
d['user_id'] = df.iloc[0]['user_id']
d['movie_id'] = df.iloc[0]['movie_id']
d['user_id'] = 
d

{'user_id': 1, 'movie_id': 1193}

In [157]:
test_set = []
val_set = []
# to_numpy()는 벡터나 행렬을 ndarray형태로 바꿔줌
# nonzero()는 0이 아닌 값들의 인덱스를 알려줌. idx는 row index, jdx는 column index라 생각하면 됨
idx, jdx = sparse_matrix.fillna(0).to_numpy().nonzero()
indice = list(zip(idx, jdx))
print(len(indice))
np.random.shuffle(indice)
for i,j in indice[:q.shape[0] // 10]:
    test_set.append((i, j, sparse_matrix.iloc[i,j]))
    sparse_matrix.iloc[i,j] = 0
    
for i,j in indice[q.shape[0] // 10 : q.shape[0] // 5]:
    val_set.append((i, j, sparse_matrix.iloc[i,j]))
    sparse_matrix.iloc[i,j] = 0
    
print(len(test_set))
len(val_set)

1000209
100020


100021

In [154]:
len(indice)

600127

In [95]:
sparse_matrix

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,NaN,NaN,NaN,NaN,4.0,NaN,0.0,5.0,5.0,...,NaN,4.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
I = np.random.normal(scale=1./16, size=(20,16))
U = np.random.normal(scale=1./16, size=(10,16))
I[5,:]

array([ 0.04565188, -0.04136567,  0.0824889 ,  0.0391838 , -0.02112112,
        0.00904026,  0.03249421, -0.071325  , -0.08235581, -0.05260899,
        0.03849356,  0.03407682, -0.08009137, -0.07015213,  0.01438631,
        0.04717213])

In [147]:
i_bias = np.zeros(20)
u_bias = np.zeros(10)
pred_matrix = i_bias[:,np.newaxis] + u_bias[np.newaxis:,] + I.dot(U.T)
pred_matrix[15,5]

-0.003450142170003164

In [50]:
df = pd.read_csv('ratings_Beauty.csv', encoding='utf-8')
df.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [57]:
userlist = []
itemlist = []
user_id = 0
item_id = 0

f = open('user_list.txt', 'w')
for i in range(len(df.index)):
    if df['UserId'][i] not in userlist:
        userlist.append(df['UserId'][i])

for i in range(len(userlist)):
    f.write(str(userlist[i]))
    f.write(' ')
    f.write(str(i))
    f.write('\n')
f.close()

f = open('item_list.txt', 'w')
for i in range(len(df.index)):
    if df['ProductId'][i] not in itemlist:
        itemlist.append(df['ProductId'][i])
        
for i in range(len(itemlist)):
    f.write(str(itemlist[i]))
    f.write(' ')
    f.write(str(i))
    f.write('\n')
f.close()